<div class="alert alert-success" style = "border-radius:10px;border-width:3px;border-color:white;font-family:Verdana,sans-serif;font-size:16px;">
<h2> Improved models implementation</h2>

In this Jupyter notebook, which is part of the 'Models' folder, we focus on constructing queries for the improved models, both in Spanish and English. The process begins by requesting concise outputs, followed by adding evaluation criteria, and finally appending the conversation text to each query. This is done for each conversation individually. We then set up an endpoint to the deployed model and send the queries, continuing until we receive an answer containing either '1' or '0' but not both or neither. If a query is repeated five times without yielding an answer, we record a '2' for that item.

After evaluating all items of a conversation, we repeat the querying process two more times. If there is any divergence among the three outputs, we mark the output as '3' to indicate uncertainty. The final results are saved into a .csv file for detailed analysis.

To accommodate potential breakdowns during execution, particularly considering AWS's session expiration, we have incorporated a safeguard mechanism. Each time we receive an output vector, we delete the corresponding transcribed text from the folder to prevent redundancy and add the output vector to the .csv. This approach ensures the smooth continuation of the process and maintains the integrity of the data.

In [62]:
import json
import boto3
import csv
import os
from collections import Counter

In [63]:
def evaluate_acogidaCorporativa (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Uses a greeting and corporate presentation according to the channel and whether it's an incoming or outgoing call.
*If the interlocutor changes during the conversation, they introduce themselves again.
*Continues with the presentation protocol even if the client responds to their greeting.
*Starts the presentation within 3 seconds of receiving the call.
*Is prepared for the next call. Gives a positive impression. Their presentation is natural, understandable, and professional.
*The client interrupts the advisor's presentation to directly expose their point.
*Due to reasons beyond the advisor's control, the presentation cannot be understood.
Assign 0 if it meets the following:
*Does not use a corporate presentation.
*Informal greeting, lack of concentration.
*If the interlocutor changes during the conversation, they do not reintroduce themselves.
*Shows signs of tiredness, apathy in the greeting, repetitiveness. Their presentation is mechanical, hard to understand, and/or not very professional.
Conversation:
{conversation}
"""

    return output_string

def evaluate_identificaciónDelCliente (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Correctly identifies the client or interlocutor.
*Asks and confirms all the data established by the security protocol.
*Does not provide information to non-policyholders.
*Does not offer information that for security reasons (LOPD) Generali does not wish to disclose.
Assign 0 if it meets the following:
*Does not identify the client or interlocutor.
*Does not adhere to the security policy.
*Provides information to a non-policyholder.
*Offers information that for security reasons Generali does not wish to disclose.
Conversation:
{conversation}
"""

    return output_string

def evaluate_despedidaCorporativa  (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Before ending the call, asks the client if they can offer additional help, using phrases like "Can I help you with anything else?" or "Can I provide you with any additional information?"
*When the call is cut off.
Assign 0 if it meets the following:
*Does not offer additional help.
Conversation:
{conversation}
"""

    return output_string

def evaluate_ofreceAyudaAdicional (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Uses the established farewell for each channel and service.
*The farewell is natural and understandable.
*When the call is cut off.
Assign 0 if it meets the following:
*Does not use the established farewell.
*Uses an informal farewell.
*The farewell is mechanical and difficult to understand.
*Does not say goodbye, remaining silent at the end of the call.
Conversation:
{conversation}
"""

    return output_string

def evaluate_correcciónGramatical  (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*It is expressed in a grammatically correct manner.
*It does not use "dequeísmos," "laísmos," "leísmos,"...
Assign 0 if it meets the following:
*It uses dequeisms,laisms and leisms
*It uses grammatically incorrect expressions.
Conversation:
{conversation}
"""

    return output_string

def evaluate_expresiónOral (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*It is expressed in a clear, precise, organized, concise, and formal manner, adapting to the client's level.
*Demonstrates verbal fluency and ease in using language throughout the conversation.
Assign 0 if it meets the following:
*It is expressed imprecisely and disorganized.
*Uses filler words, unclear terms, or clichéd phrases.
*Uses words or expressions that the interlocutor does not understand.
*Does not demonstrate verbal fluency.
Conversation:
{conversation}
"""

    return output_string

def evaluate_Vocabulario (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Uses a rich, correct, positive, and commercial vocabulary.
*Omits filler words, slang, negative words, diminutives, superlatives, imprecise words, etc.
*Does not use internal procedure terms or technical jargon.
*Expresses themselves in a clear, precise, organized, concise, and formal manner, adapting to the client's level.
*Demonstrates verbal fluency and ease in using language throughout the conversation.
Assign 0 if it meets the following:
*Their vocabulary is poor, incorrect, negative, or not suitable for a commercial context.
*Uses: Negative words (problem), Negative terms (no), Dubitative words (maybe), Imprecise words (maybe, more or less...)
Conversation:
{conversation}
"""

    return output_string

def evaluate_entonaciónModulación (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Avoids monotony, varies the pace, and uses expressiveness.
*Modulates and intonates the voice, adapting it to each of the phrases in the conversation.
*Maintains a consistent medium volume, neither shouting nor whispering.
Assign 0 if it meets the following:
*Does not modulate (high/low), is monotonous, tiresome, apathetic, lacks musicality, etc., making the interlocutor have to make an effort to understand the message conveyed by the advisor.
*Shouts or whispers, increases the volume to emphasize the conversation.
Conversation:
{conversation}
"""

    return output_string

def evaluate_correctaArticulaciónElocuciónAdecuada (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Avoids monotony, varies the rhythm, and uses expressiveness.
*Modulates and intonates the voice, adapting it to each of the phrases in the conversation.
*Maintains a consistent medium volume, neither shouting nor whispering.
Assign 0 if it meets the following:
*Does not modulate (highs/lows), is monotonous, tedious, apathetic, lacks musicality, etc., causing the interlocutor to make an effort to understand the message conveyed by the advisor.
*Shouts or whispers, increases the volume to emphasize.
Conversation:
{conversation}
"""

    return output_string

def evaluate_sonrisaTelefónica (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Uses a smile whenever possible.
*Maintains a positive attitude and a cheerful tone.
Assign 0 if it meets the following:
*The situation allows for a smile, but it is not perceived.
Conversation:
{conversation}
"""

    return output_string

def evaluate_FormulasDeCortesía (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Clear signs of deference and respect for the interlocutor are evident.
*Uses polite expressions during the conversation and when questioning the client.
*Apologizes on those occasions when any inconvenience or harm has been caused to the client.
Assign 0 if it meets the following:
*Does not use polite expressions during the conversation and/or when questioning the client.
*Uses imperatives.
*Does not apologize on those occasions when any inconvenience or harm has been caused to the client.
Conversation:
{conversation}
"""

    return output_string

def evaluate_noInterrumpir (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Clear signs of deference and respect for the interlocutor are evident.
*Uses polite expressions during the conversation and when questioning the client.
*Apologizes on those occasions when any inconvenience or harm has been caused to the client.
Assign 0 if it meets the following:
*Does not use polite expressions during the conversation and/or when questioning the client.
*Uses imperatives.
*Does not apologize on those occasions when any inconvenience or harm has been caused to the client.
Conversation:
{conversation}
"""

    return output_string

def evaluate_Esperas (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
a) Justifies the wait:
*Informs the client to keep them on hold.
*Uses hold times correctly, only when necessary and in cases of consulting with a supervisor or transferring the call.
*When needing to transfer the call, informs the client in advance and uses the correct argument to explain the transfer.
*Justifies the reason for the hold with the correct argument.
Assign 0 if it meets the following:
*Does not inform the client to keep them on hold.
*Does not justify the hold, does not inform the need for it in advance, and does not provide a valid argument for it.
b) Retakes the call:
*When putting the client on hold, resumes the call every 30 seconds.
*Uses the correct argument each time when resuming the call, informing the client of the progress.
*Offers apologies for keeping them on hold and resumes following the established protocol (Mr. Martínez, thank you for your patience).
*When there is no need to put the interlocutor on hold at any point.
Assign 0 if it meets the following:
*Does not resume the call periodically, every 30 seconds.
*Does not resume correctly, does so without thanking for the wait and without providing any explanation, going directly to the solution.
*When needing to transfer the call, does not inform the client and/or uses an incomplete/incorrect argument to explain the transfer.
Conversation:
{conversation}
"""

    return output_string

def evaluate_controlDeLosSilencios (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Avoids unnecessary silences.
*Avoids the "dark tunnel effect": staying without an argument and not knowing how to respond to the client, keeping them on hold for a long time while managing without resuming and justifying the wait.
Assign 0 if it meets the following:
*Unnecessary silences throughout the conversation.
*Remains silent without arguments or experiences prolonged silences while managing without justification.
Conversation:
{conversation}
"""

    return output_string

def evaluate_escuchaActiva (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Shows interest and attentiveness to what is being discussed, adapts to the psychology of the interlocutor.
*Understands any attitude.
*Makes the interlocutor feel that they are paying the utmost attention to their explanations.
*Intervenes to avoid awkward silences during the conversation.
*Every time active listening is practiced, it is done naturally.
Assign 0 if it meets the following:
*No effort or interest is perceived in paying attention to the explanations of the interlocutor.
*Does not understand reactions and responds in the same manner.
*Awkward silences occur during the conversation.
*Every time active listening is practiced throughout the conversation, it is done mechanically.
Conversation:
{conversation}
"""

    return output_string

def evaluate_necesidadDelCliente (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Understands the interlocutor's position, shows absolute empathy.
*Availability to provide assistance.
*Conducts proper inquiry (reception) / Message presentation (transmission).
*Asks key questions to gather the necessary information to handle the call.
*Restates information provided by the client.
*Reconstructs and summarizes agreements or data. This is done throughout the conversation.
Assign 0 if it meets the following:
*Displays excessive indifference and coldness.
*Does not make an effort to inquire about what is presented, only investigates the bare minimum.
*Does not offer different alternatives to the topic raised or guide the client towards the one that best suits their needs.
*Does not present information in a structured manner.
*Does not repeat agreements.
*Does not clarify vague ideas.
Conversation:
{conversation}
"""

    return output_string

def evaluate_personalizaciónEnElTrato (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Addresses the person as a known interlocutor (not just anyone).
*Once the interlocutor is identified (whether or not they are the account holder), addresses them using the title Mr./Mrs. followed by the last name or Mr./Ms. followed by the first name. If the last name is offensive or difficult to pronounce, the title Mr./Ms. will be used.
*Whenever resuming a call or needing to get the interlocutor's attention, uses the established protocol (Mr./Mrs. Mr./Ms.).
*Always uses the formal "Usted" form of address when addressing the client unless the client expressly requests otherwise.
Assign 0 if it meets the following:
*Gives the impression of not knowing who they are speaking to.
*After identifying the interlocutor, does not use the appropriate form of address.
*When resuming a call or getting the interlocutor's attention, does not use the correct form of address, but rather uses an incorrect and/or informal one.
*Uses the informal "tú" form of address with the client when not requested.
Conversation:
{conversation}
"""

    return output_string

def evaluate_direccionDeLaLlamada (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Knows how to direct the call.
*Maintains control of the conversation, presents alternatives.
*Adopts a confident attitude, leading the conversation so that the client has complete trust in the service. Firmness and assurance in statements.
*Confidence in reactions and responses.
*Changes the client's negative perception through arguments.
Assign 0 if it meets the following:
*Does not direct the call, with the client setting the guidelines during the conversation.
*Does not propose alternatives.
*Monologues are penalized.
*Conveys insecurity, making the client not fully trust the service.
*Uses hesitant expressions or words repeatedly.
*Frequently hesitates, leaving sentences unfinished.
*Fails to convince.
*Brief silences that show nervousness or insecurity.
*Constantly consults with each question posed by the client.
*Excessive confidence is also considered incorrect, i.e., when the advisor adopts a presumptuous and aggressive attitude towards the client.
*Poor handling of objections. Does not know how to handle objections. Does not counter them or does so without conviction, being hesitant, imprecise, etc.
*Remains silent in the face of client objections.
*Maintains a negative tone with the client.
Conversation:
{conversation}
"""

    return output_string

def evaluate_imagenDeLaEmpresa (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Provides information, a positive image of the company, and its products and services.
*Projects a positive image of the company, seeking both the client's and the company's benefit.
*Assumes responsibility for any errors made by colleagues as if they were their own.
Assign 0 if it meets the following:
*Creates a negative image of the company.
*Shows indifference to the benefit of the company and the client, projecting a negative image of the company.
*Makes excuses blaming colleagues or other departments.
Conversation:
{conversation}
"""

    return output_string

def evaluate_empatia (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Understands the interlocutor's position, absolute empathy.
Assign 0 if it meets the following:
*Reveals excessive indifference and coldness.
Conversation:
{conversation}
"""

    return output_string

def evaluate_correctoUsoAplicacionesSistemas (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Uses different applications correctly for consulting or handling customer requests.
*Incidents in the systems.
Assign 0 if it meets the following:
*Does not use the available applications correctly.
*Does not transfer correctly (when necessary).
Conversation:
{conversation}
"""

    return output_string

def evaluate_conocimientoAplicaciónProtocolosProcedimientos (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Knows the procedures and protocols and applies them correctly during the call and/or handling of requests.
Assign 0 if it meets the following:
*Does not know or does not apply the established procedures for each call, type of request, etc.
Conversation:
{conversation}
"""

    return output_string

def evaluate_clasificaciones (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Classifies the call correctly in the application, and the classification must match the content of the call (including call observations in the classification).
*Incidents in the systems.
Assign 0 if it meets the following:
*Does not classify the call or classifies it incorrectly.
Conversation:
{conversation}
"""

    return output_string

def evaluate_resolucion (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""The response should only be 1 or 0. Give a 1 if the following conversation meets these criteria:
*Resolves a situation that can conclude on its own, avoiding subsequent calls from the client for the same reason.
*If the situation cannot be resolved online, transfers the call/management to the correct department following the established protocol.
*Confirms the action taken with the client.
*Provides a final reformulation summarizing everything managed during the call.
*When unable to resolve during the call due to technical system issues.
Assign 0 if it meets the following:
*Does not make an effort to find a solution within their reach.
*Does not verify the client's understanding.
*Displays a passive, indifferent, or unconcerned attitude.
*Does not confirm with the client the action taken.
*Does not provide any final reformulation or summary of the agreements reached with the client.
Conversation:
{conversation}
"""

    return output_string

In [64]:
def evaluate_acogidaCorporativa (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Emplea saludo y presentación corporativa según canal y emisión / recepción
* Si en el transcurso de la conversación cambia el interloculor se presenta nuevamente
* Continúa con el protocolo de presentación aunque el cliente conteste a su saludo
* Comienza la presentación en un tiempo inferior a 3 seg tras recibir la llamada
* Está preparado para la siguiente llamada. Sensación positiva. Su presentación es natural, entendible y comercial
* El cliente interrumpe la presentación del asesor para exponer directamente el argumento
* Por causas ajenas al asesor no se pueda entender la presentación
0 si cumple:
* No emplea presentación corporativa
* Saludo coloquial, falta de concentración.
* Si el transcurso de la conversación cambia el interloculor no vuelve a presentarse
* Denota cansancio, apatía en el saludo, repetitividad. Su presentación es mecánica, difícil de entender y/o poco comercial.
conversación:
{conversation}
"""

    return output_string

def evaluate_identificaciónDelCliente (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Identifica de forma correcta al cliente o interlocutor.
* Pregunta y confirma todos los datos establecidos por protocolo en la política de seguridad
* No facilitar información a no tomadores de las pólizas
* No ofrece informacion que por motivos de seguridad (LOPD) Generali no desea divulgar
0 si cumple:
* No identifica al cliente o interlocutor
* No pasa pólitica de seguridad 
* Facilita informacion a no tomador
* Ofrece información que por motivos de seguridad Generali no desea divulgar
conversación:
{conversation}
"""

    return output_string

def evaluate_despedidaCorporativa  (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Antes de finalizar la llamada pregunta al cliente si puede ofrecerle ayuda adicional con fórmulas como ¿puedo ayudarle en algo más? ¿puedo facilitarle alguna información adicional?
* Cuando se corta la llamada
0 si cumple:
* No ofrece ayuda adicional
conversación:
{conversation}
"""

    return output_string

def evaluate_ofreceAyudaAdicional (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Utiliza la despedida/s establecida para cada canal y servicio
* Su despedida es natural y entendible           
* Cuando se corta la llamada
0 si cumple:
* No utiliza la despedida establecida.
* Despedida coloquial.
* Su despedida es mecánica y difícil de entender.
* No se despide, permaneciendo en silencio al finalizar la llamada.
conversación:
{conversation}
"""

    return output_string

def evaluate_correcciónGramatical  (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Se expresa de forma gramaticalmente correcta
* No emplea dequeísmos, laísmos, leísmos,…
0 si cumple:
* Utiliza dequeísmos, laísmos, leísmos,… 
* Emplea expresiones gramaticalmente incorrectas
conversación:
{conversation}
"""

    return output_string

def evaluate_expresiónOral (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Se expresa de forma clara, precisa, ordenada, concisa y formal, adaptándose al nivel del cliente. 
* Demuestra tener fluidez verbal y facilidad para emplear el lenguaje durante el transcurso de la conversación
0 si cumple:
* Se expresade forma imprecisa y desordenada. 
* Utiliza palabras de relleno, poco claras o frases hechas. 
* Utiliza palabras o expresiones que el interlocutor no entiende. 
* No demuestra tener fluidez verbal
conversación:
{conversation}
"""

    return output_string

def evaluate_Vocabulario (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Utiliza un vocabulario rico, correcto, positivo, comercial.
* Omite muletillas, jerga, palabras negras, negativas, diminutivos, superlativos, palabras imprecisas, etc. 
* No emplea términos de procedimientos internos o tecnicísmos* Se expresa de forma clara, precisa, ordenada, concisa y formal, adaptándose al nivel del cliente. 
* Demuestra tener fluidez verbal y facilidad para emplear el lenguaje durante el transcurso de la conversación
0 si cumple:
* Su vocabulario es pobre, incorrecto, negativo o poco comercial. 
* Utiliza: Palabras negras (problema), Palabras negativas (no), Palabras dubitativas (quizás), Palabras imprecisas (quizás, más o menos...),  Muletillas, argot, coloquialismos, Frases hechas, Diminutivos (un momentito), Superlativos, Tecnicismos y  expresiones propias del departamento.
conversación:
{conversation}
"""

    return output_string

def evaluate_entonaciónModulación (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Evita la monotonía, varia el ritmo, expresividad.
* Modula y entona la voz adaptándola a cada una de las frases de la conversación
* Conserva siempre un volumen medio, ni grita ni susurra
0 si cumple:
* No modula (altos/bajos), es monocorde, cansino, apático, tiene musicalidad, etc., lo que provoca que el interlocutor tenga que realizar un esfuerzo para comprender el mensaje emitido por el asesor
* Grita o susurra, sube el volumen para enfatizar
conversación:
{conversation}
"""

    return output_string

def evaluate_correctaArticulaciónElocuciónAdecuada (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Evita la monotonía, varia el ritmo, expresividad.
* Modula y entona la voz adaptándola a cada una de las frases de la conversación
* Conserva siempre un volumen medio, ni grita ni susurra
0 si cumple:
* No modula (altos/bajos), es monocorde, cansino, apático, tiene musicalidad, etc., lo que provoca que el interlocutor tenga que realizar un esfuerzo para comprender el mensaje emitido por el asesor
* Grita o susurra, sube el volumen para enfatizar
conversación:
{conversation}
"""

    return output_string

def evaluate_sonrisaTelefónica (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Emplea la sonrisa siempre que es posible. 
* Mantiene una actitud positiva, un tono alegre.
0 si cumple:
* La situación permite sonrisa y no se percibe
{conversation}
"""

    return output_string

def evaluate_FormulasDeCortesía (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Aparecen claras muestras de deferencia y respeto por el interlocutor. 
* Hace uso de fórmulas de cortesía durante el transcurso de la conversación y al interrogar al cliente. 
* Pide disculpas en aquellas ocasiones en las que se ha ocasionado alguna molestica o perjuicio al cliente
0 si cumple:
* No hace uso de fórmulas de cortesía durante el transcurso de la conversación y/o al interrogar al cliente . 
* Empleo de imperativos. 
* No pide disculpas en aquellas ocasiones en las que se ha ocasionado alguna molestio o perjuicio al cliente
{conversation}
"""

    return output_string

def evaluate_noInterrumpir (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Aparecen claras muestras de deferencia y respeto por el interlocutor. 
* Hace uso de fórmulas de cortesía durante el transcurso de la conversación y al interrogar al cliente. 
* Pide disculpas en aquellas ocasiones en las que se ha ocasionado alguna molestica o perjuicio al cliente
0 si cumple:
* No hace uso de fórmulas de cortesía durante el transcurso de la conversación y/o al interrogar al cliente . 
* Empleo de imperativos. 
* No pide disculpas en aquellas ocasiones en las que se ha ocasionado alguna molestio o perjuicio al cliente
{conversation}
"""

    return output_string

def evaluate_Esperas (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
a) Justifica la espera:
* Se avisa al cliente para mantenerle en espera 
* Utiliza las esperas de forma correcta, sólo si es necesario y en los casos de consulta al supervisor o transferencia de la llamada. 
* Cuando tiene que transferir la llamada, informa previamente de ello y emplea el argumento correcto para informar de la misma. 
* Justifica el motivo de la espera con el argumento correcto
b) Retoma la llamada:
* Cuando tiene que dejar al cliente en espera, retoma la llamada cada 30 seg
* Emplea un argumento correcto cada vez que retoma al cliente informándole de la gestión
* Presenta disculpas por haberle mantenido en espera y retoma siguiendo el protocolo establecido (Sr. Martínez, gracias por su espera) * Cuando no se da la circunstancia de dejar al interlocutor en espera en ningún momento.
0 si cumple:
a) Justifica la espera:
* No informa al cliente para mantenerle a la espera. 
* No justifica la espera, no informa previamente de la necesidad de la msma ni justifica el motivo con un argumento correcto.
b) Retoma la llamada:
* No retoma la llamada periódicamente, cada 30 seg.
* No retoma de forma correcta, lo hace sin agradecer la espera y sin ningún argumento, exponiendo directamente la solución.
* Cuando tiene que transferir no informa y/o emplea un argumento incompleto/incorrecto para informar de la transferencia.
{conversation}
"""

    return output_string

def evaluate_controlDeLosSilencios (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Evita silencios innecesarios.
* Evita el efecto "tunel oscuro": nos quedamos sin argumento y no sabemos que contestar al cliente, mantenemos en espera mucho tiempo mientras gestionamos sin retomar y justificar la espera
0 si cumple:
* Silencios innecesarios a lo largo de la conversación.
* Se queda en silencio y sin argumentos, o se mantiene en silencio prolongado mientras gestiona sin justificar
{conversation}
"""

    return output_string

def evaluate_escuchaActiva (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Manifiesta interés y atención por lo que le plantean, se adapta a la psicología del interlocutor.
* Comprende cualquier actitud. 
* Hace sentir al interlocutor que está prestando la máxima atención a sus explicaciones. 
* Interviene evitando incómodos silencios durante el transcurso de la conversación. 
* Cada vez que practica la escucha activa lo hace de forma natural.
0 si cumple:
* No se percibe esfuerzo en la atención ni interés por las explicaciones del interlocutor. 
* No entiende las reacciones y replica con las mismas formas. 
* Se producen incómodos silencios durante el transcurso de la conversación. 
* Cada vez que practica la escucha activa a lo largo de la conversación lo hace de forma mecánica.
{conversation}
"""

    return output_string

def evaluate_necesidadDelCliente (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Entiende la posición del interlocutor, empatía absoluta
* Disponibilidad para la ayuda
* Realiza un correcto sondeo (recepción) / Exposición del mensaje (emisión)
* Hace preguntas claves para obtener la información necesaria para dirigir la llamada.
* Reformula datos aportados por el cliente
* Reconstruye y sintetiza acuerdos o datos. Lo hace a lo largo de la conversación
0 si cumple:
* Revela excesiva indiferencia y frialdad
* No se esfuerza en averiguar lo que le plantean, sólo indaga lo mínimo. 
* No ofrece distintas alternativas al tema planteado ni orienta al cliente hacia la que más se adecua a sus necesidades. 
* No expone de forma estructurada.
* No repite los acuerdos.
* No aclara las ideas difusas
{conversation}
"""

    return output_string

def evaluate_personalizaciónEnElTrato (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Se dirige a la persona como un interlocutor conocido (no es uno más). 
* Una vez identificado el interlocutor (sea o no el titular) se dirige a él utilizando el tratamiento Sr./Sra. seguido del apellido o D./Dña. seguido del nombre. En caso de que el apellido sea malsonante o difícil de pronunciar se optará por el tratamiento D/Dña. 
* Siempre que retoma una llamada o tiene que llamar la atención del interlocutor se dirige utilizando el protocolo establecidido (Sr/Sra. D/Dña). 
* Siempre utiliza el tratamiento de Usted al dirigirse al cliente, salvo que expresamente lo solicite el cliente
0 si cumple:
* Da la impresión de no saber con quien habla.
* Identificado el interlocutor no hace uso del tratamiento correspondiente. 
* Al retonar la llamada o llamar la atención del interlocutor no lo hace utilizando el tratamiento correspondiente, sino de forma incorrecta y/o coloquial
* Tutea al cliente cuando el no lo solicite.
{conversation}
"""

    return output_string

def evaluate_direccionDeLaLlamada (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Sabe dirigir la llamada
* Lleva el control de la conversación, plantea alternativas
* Adopta una actitud segura, dirigiendo la conversación con el fin de que el cliente confie plenamente en el servicio. Firmeza y seguridad en las afirmaciones
* Seguridad en las reacciones y respuestas
* Cambia la percepción negativa del cliente a través de argumentos.
0 si cumple:
* No dirige la llamada siendo el cliente el que establece las pautas durante la conversación. 
* No propone alternativas.
* Se penalizan los monólogos
* Transmite inseguridad y hace que el cliente no confie plenamente en el servicio
* Utiliza expresiones o palabras dubitativas de forma reiterada.
* Titubea dejando frases a medias de forma constante.
* No logra convencer.
* Silencios breves que muestran nervios o inseguridad.
* Consulta constantemente cada pregunta que le plantea el cliente.
* También se considera incorrecto el exceso de seguridad, es decir, cuando el asesor adopta una actitud prepotente y agresiva hacia al cliente
* Mal tratamiento de objeciones. No sabe tratar las objeciones. No las rebate o lo hace sin convicción, siendo dubitarivo, impreciso, etc. 
* Permanece en silencio ante las objecciones del cliente.
* Mantiene el tono negativo de un cliente.
{conversation}
"""

    return output_string

def evaluate_imagenDeLaEmpresa (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Aporta información, buena imagen de la compañía y de sus productos y servicios. 
* Proyecta una imagen positiva de la empresa, buscando tanto el beneficio del cliente como el de la empresa. 
* Asume las responsabilidades derivadas del posible error de cualquier compañero como si hubiera sido propio.
0 si cumple:
* Crea una mala imagen de la empresa. 
* Se muestra indiferente ante el beneficio de Generali y del cliente, proyectando una imagen negativa de la empresa.
* Se excusa en los compañeros o en otros departamentos
{conversation}
"""

    return output_string

def evaluate_empatia (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Entiende la posición del interlocutor, empatía absoluta
0 si cumple:
* Revela excesiva indiferencia y frialdad
{conversation}
"""

    return output_string

def evaluate_correctoUsoAplicacionesSistemas (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Utiliza las diferentes aplicaciones de forma correcta para su consulta o la gestión de solicitudes de los clientes.
* Incidencias en los sistemas
0 si cumple:
* No realiza un uso correcto de las aplicaciones disponibles.
* No transfiere de forma correcta (cuándo proceda). 

{conversation}
"""

    return output_string

def evaluate_conocimientoAplicaciónProtocolosProcedimientos (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Conoce los procedimientos y protocolos y los aplica de forma correcta en la llamada y/o gestión.
0 si cumple:
* No conoce o no aplica los procedimientos establecidos para cada llamada, tipo de solicitud, etc. 

{conversation}
"""

    return output_string

def evaluate_clasificaciones (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Tipifica la llamada y lo hace de forma correcta en el aplicativo. La tipificación debe ajustarse al contenido de la llamada (incluyendo observaciones de la llamada en la tipificación)
* Incidencias en los sistemas
0 si cumple:
* No tipifica o tipifica de forma incorrecta la llamada

{conversation}
"""

    return output_string

def evaluate_resolucion (file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation = file.read() 
        
    output_string = f"""La respuesta unicamente debe ser 1 o 0 Sea 1 si el siguiente conversación cumple:
* Resuelve una situación que puede concluir por si mismo evitando llamadas posteriores del cliente por el mismo motivo
* Si la situación no puede resolverse online, transfiere la llamada / gestión al departamento correcto siguiendo el protocolo marcado
* Confirma con el cliente la gestión realizada
* Hace una reformulación final resumen de todo lo gestionado en la llamada.                                            
* Cuando por incidencia técnica en los sistemas no puede resolver en la llamada
0 si cumple:
* No se esfuerza en encontrar una solución a su alcance
* No verifica la comprensión del cliente
* Actitud pasiva, indiferente, despreocupación
* No confirma con el cliente la gestión realizada
* No realiza ninguna reformulación final, resumen de los acuerdos alcanzados con el mismo

{conversation}
"""

    return output_string

In [65]:
def createQuery(file_path):
    dialogs = [
        evaluate_acogidaCorporativa(file_path),
        evaluate_identificaciónDelCliente(file_path),
        evaluate_despedidaCorporativa(file_path),
        evaluate_ofreceAyudaAdicional(file_path),
        evaluate_correcciónGramatical(file_path),
        evaluate_expresiónOral(file_path),
        evaluate_Vocabulario(file_path),
        evaluate_entonaciónModulación(file_path),
        evaluate_correctaArticulaciónElocuciónAdecuada(file_path),
        evaluate_sonrisaTelefónica(file_path),
        evaluate_FormulasDeCortesía(file_path),
        evaluate_noInterrumpir(file_path),
        evaluate_Esperas(file_path),
        evaluate_controlDeLosSilencios(file_path),
        evaluate_escuchaActiva(file_path),
        evaluate_necesidadDelCliente(file_path),
        evaluate_personalizaciónEnElTrato(file_path),
        evaluate_direccionDeLaLlamada(file_path),
        evaluate_imagenDeLaEmpresa(file_path),
        evaluate_empatia(file_path),
        evaluate_correctoUsoAplicacionesSistemas(file_path),
        evaluate_conocimientoAplicaciónProtocolosProcedimientos(file_path),
        evaluate_clasificaciones(file_path),
        evaluate_resolucion(file_path)]
        
    return dialogs

In [66]:
endpoint_name = "jumpstart-dft-meta-textgeneration-llama-2-7b-f"


def query_endpoint(payload):
    client = boto3.client("sagemaker-runtime")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=json.dumps(payload),
        CustomAttributes="accept_eula=true",
    )
    response = response["Body"].read().decode("utf8")
    response = json.loads(response)
    return response

In [67]:
def generate_output_vector(dialogs):
    output_vector = []  # Initialize an empty list to store the output

    for dialog in dialogs:
        contains_1 = False
        contains_0 = False
        
        i = 0
        while not (contains_1 or contains_0):
            payload = {
                "inputs": dialog, 
                "parameters": {"max_new_tokens": 500, "top_p": 0.9, "temperature": 0.6}
            }
            result = query_endpoint(payload)[0]
            generated_content = result['generated_text'][:300]  # Extract the first 300 characters
            contains_1 = "1" in generated_content
            contains_0 = "0" in generated_content

            if i == 5:  # Break the loop and append '2' after 5 attempts
                output_vector.append(2)
                break
            i += 1
            if contains_1 and contains_0:
                contains_1 = False
                contains_0 = False
                
        else:  # Append '1' or '0' if found in the generated content
            output_vector.append(1 if contains_1 else 0)

    return output_vector

In [ ]:

# Specify the directory containing the transcribed files
directory_path = "../output"

# Get a list of all files in the specified directory
all_files = [file for file in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, file))]

# Process each file
for file_name in all_files:
    file_path = os.path.join(directory_path, file_name)
    dialogs = createQuery(file_path)
    result_vector1 = generate_output_vector(dialogs)
    result_vector2 = generate_output_vector(dialogs)
    result_vector3 = generate_output_vector(dialogs)
    
    # Modify the merging strategy
    merged_vector = []
    for i in range(len(result_vector1)):
        if result_vector1[i] == result_vector2[i] == result_vector3[i]:  # All three vectors have the same value
            merged_vector.append(result_vector1[i])
        else:
            merged_vector.append(3)  # Different values in vectors, append 3
    
    
    # Create a CSV file to write the results
    csv_file_path = "output_results_baseline_model_esp_improvemtns.csv"
    with open(csv_file_path, mode='a', newline='') as csv_file:
        fieldnames = ['File', 'Result']  # Define the column headers
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        # Check if the file is empty, and write the header row if needed
        if os.path.getsize(csv_file_path) == 0:
            writer.writeheader()

        # Write the result to the CSV file
        writer.writerow({'File': file_name, 'Result': merged_vector})

    # Delete the processed file
    os.remove(file_path)